# Library

In [1]:
import pandas as pd
import requests
import json
import time
import re

In [13]:
# Function to unload the model
def unload_model(url, model_name):
    unload_payload = {
        "model": model_name,
        "keep_alive": 0  # This tells the API to unload the model
    }

    # Send the POST request to unload the model
    response = requests.post(url, json=unload_payload)

    if response.status_code == 200:
        print(f"Model '{model_name}' has been unloaded successfully.")
    else:
        print(f"Failed to unload model '{model_name}'. Error: {response.status_code}, {response.text}")

# Function to load the model
def load_model(url, model_name):
    load_payload = {
        "model": model_name  # No `keep_alive` to indicate loading
    }

    # Send the POST request to load the model
    response = requests.post(url, json=load_payload)

    if response.status_code == 200:
        print(f"Model '{model_name}' has been loaded successfully.")
    else:
        print(f"Failed to load model '{model_name}'. Error: {response.status_code}, {response.text}")

# Main function to unload and load the model
def reload_model(url, model_name):
    # Unload the model
    unload_model(url, model_name)
    
    # Load the model
    load_model(url, model_name)

import requests

def chat_to_model(chat_url, model, user_input, stream=False):
    """
    Sends a user message to the assistant model and retrieves the response.
    
    Args:
        chat_url (str): The URL of the chat API endpoint.
        model (str): The name of the model to interact with.
        user_input (str): The input message to send to the model.
        stream (bool): Whether to enable streaming mode for the model.
        
    Returns:
        str: The assistant's response, or an error message if the request fails.
    """
    
    # Prepare the payload for the POST request
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": user_input
            }
        ],
        "stream": stream
    }
    
    # Send the POST request to the API
    response = requests.post(chat_url, json=payload)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response content
        response_json = response.json()
        response_content = response_json.get("message", {}).get("content", "No content found.")
        return response_content
    else:
        return f"Error: {response.status_code}, {response.text}"
    

def send_conversation_to_model(chat_url, model, conversation):
    """
    Sends a conversation (multiple messages) to the assistant model and retrieves the response.
    
    Args:
        chat_url (str): The URL of the chat API endpoint.
        model (str): The name of the model to interact with.
        conversation (list): A list of dictionaries containing 'role' and 'content' keys.
        
    Returns:
        str: The assistant's response, or an error message if the request fails.
    """
    
    # Prepare the payload for the POST request
    payload = {
        "model": model,
        "messages": conversation,
        "format": "json",
        "options": {
            "seed": 111,
            "num_ctx": 10240
        },
        "stream": False
    }
    
    # Send the POST request to the API
    response = requests.post(chat_url, json=payload)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response content
        response_json = response.json()
        response_content = response_json.get("message", {}).get("content", "No content found.")
        return response_content
    else:
        return f"Error: {response.status_code}, {response.text}"

In [14]:
# Define content slicing method
def sample_content(content, max_length=1750):
    if len(content) <= max_length:
        return content
    
    # Calculate positions for start, middle, and end chunks
    start = content[:350]
    mid = content[len(content) // 2: len(content) // 2 + 700]
    end = content[-700:]
    
    # Concatenate the start, middle, and end chunks
    sampled_content = start + mid + end
    return sampled_content


def sample_content_with_keywords(content, max_length=1750, keywords=None):
    if len(content) <= max_length:
        return content
    # Define your keywords
    keywords = ['judi', 'bet', 'porn', 'sex', 'bokep', 'gacor', 'slot' ]
    # Find positions of keywords
    keyword_hits = []
    if keywords:
        for keyword in keywords:
            for match in re.finditer(keyword, content):
                keyword_hits.append(match.start())

    # If keywords are found, sample sections around the keyword positions
    if keyword_hits:
        snippets = []
        for hit in keyword_hits:
            # Get a window of text around each keyword hit
            start = max(0, hit - 100)
            end = min(len(content), hit + 100)
            snippets.append(content[start:end])
        
        # Join the snippets and cap to max_length
        sampled_content = ' '.join(snippets)[:max_length]
    else:
        # If no keywords found, fall back to sliding window sampling
        sampled_content = sample_content(content, max_length)

    return sampled_content

In [15]:
# Define the API endpoint
# chat_url = "http://localhost:11434/api/chat"
# generate_url = "http://localhost:11434/api/generate"
# chat_url = "http://112.78.144.146:11434/api/chat"
# generate_url = "http://112.78.144.146:11434/api/generate"
chat_url = "http://100.77.86.156:11434/api/chat"
generate_url = "http://100.77.86.156:11434/api/generate"

# model = "llama3.1:8b-instruct-q5_K_M"
# model = "rjmalagon/dolphin-2.9.3-mistral-nemo:q8_0"
# model = "mseri/mistral-nemo-minitron:latest"
# model = "mistral-nemo:12b-instruct-2407-q8_0"
model = "deepseek-r1:32b"

 
# Maximum length for content
MAX_CONTENT_LENGTH = 1750

In [18]:
unload_model(generate_url, model)

Model 'mseri/mistral-nemo-minitron:latest' has been unloaded successfully.


In [16]:
reload_model(generate_url, model)

Model 'deepseek-r1:32b' has been unloaded successfully.
Model 'deepseek-r1:32b' has been loaded successfully.


In [17]:
# Prepare static system prompts
with open("prompts/system_prompt.txt", "r") as file:
    system_prompt = file.read()

with open("prompts/system_prompt_answer.txt", "r") as file:
    system_prompt_answer = file.read()

with open("prompts/labelling_promptv2.txt", "r") as file:
    labelling_prompt = file.read()

with open("prompts/labelling_prompt_answer.txt", "r") as file:
    labelling_prompt_answer = file.read()

# input_suffix = "Classify the given URL as 0 (benign), 1 (gambling), or 2 (pornography) and respond with the result in JSON format.\n"
input_suffix = "EXPECTED_CLASSIFICATION = {0: \"Benign\", 1: \"Gambling\", 2: \"Pornography\"}. Respond with the result in JSON format.\n"

# Sample

In [19]:
with open(".\prompts\hallucinations\japanese.txt", "r") as file:
    input = file.read()

# contextual prompting
conversation = [
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)


'{ "answer": 0, "classification": "Benign", "reason": "The website appears to be a general communication platform with no explicit gambling or adult content." }\n \n\n \n\n \n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n'

In [20]:
with open("prompts/class_1_sample.txt", "r") as file:
    input = file.read()

conversation = [
    # {
    #     "role": "user",
    #     "content": system_prompt
    # },
    # {
    #     "role": "assistant",
    #     "content": system_prompt_answer
    # },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 1, "classification": "Gambling", "reason": "The domain \'bomjudi\' contains gambling keywords. The content lists various gambling games like Slots, Live Casino, Togel, Poker, etc., which clearly indicates it\'s a gambling website." }\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n'

In [21]:
with open("prompts/class_2_sample.txt", "r") as file:
    input = file.read()

conversation = [
    # {
    #     "role": "user",
    #     "content": system_prompt
    # },
    # {
    #     "role": "assistant",
    #     "content": system_prompt_answer
    # },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 2, "classification": "Pornography", "reason": "The domain and content include multiple categories related to explicit adult content such as \'Bokep\', which is a common term for pornography in some contexts. The website offers various types of adult material under different subcategories, clearly indicating its purpose." }\n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n \n \n\n'

In [11]:
with open("prompts/class_2_sample2.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{\n  "answer": 2,\n  "classification": "Pornography",\n  "reason": "The website https://lesbotube.net appears to contain explicit sexual content, including images and videos that are likely to be considered adult or erotic material. The categorization is based on the presence of keywords such as \'lesbian\', \'XXX fingering videos\', \'strapon\', \'orgasm\', \'big boobs\', and \'Brunette\' in the website\'s description and content."\n}\n\t\n\t\n\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t'

In [12]:
with open("prompts/class_2_sample3.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{\n  "answer": 2,\n  "classification": "Pornography",\n  "reason": "The website https://justlove.ro contains adult and erotic material, specifically a \'Vibrator Pretty Love Joyce Turquoise 18 cm\' product listed under the \'Ou Vibrator\' category. The presence of sexually explicit products suggests that the site falls into the pornography category."\n}\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t\n\t'

In [19]:
with open("prompts/arabic-farsi.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{"answer": 0, "classification": "Benign", "reason": "The website appears to be a news outlet discussing various topics including current events and politics. It does not contain content related to gambling or pornography."}'

In [20]:
with open("prompts/indonesian.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{"answer": 0,"classification": "Benign","reason": "The website https://www.tokopedia.com/ appears to be an online marketplace, which is not related to gambling or pornography. It offers a variety of products and services in different categories, including electronics, health care, travel, finance, etc."} \n\n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n '

In [38]:
with open("prompts/korean.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "Benign", "reason": "The website https://hanmaum.org appears to be a religious or spiritual organization\'s official website. It contains information about the organization\'s history, current events, and activities, as well as links to various sections such as \'법회 · 신행\' (Scriptural Meetings) and \'한마음 미디어\' (Heart-to-Heart Media). The website does not contain any explicit or mature content." }'

In [39]:
with open("prompts/english.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "benign", "reason": "The provided URL appears to be a news website, Forbes.com, featuring various articles and stories on topics such as business, finance, technology, and more. The content does not contain explicit sexual material, gambling, or any other harmful content, indicating that it is benign." }'

In [32]:
with open("prompts/japanese.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "Benign", "reason": "The website https://ihara-stadium.com appears to be a sports stadium\'s official website, providing information about events, schedules, and facilities. The content is focused on the stadium\'s operations and does not contain explicit or mature themes." }'

In [16]:
with open("prompts/chinese2.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 1, "classification": "Gambling", "reason": "The website appears to be promoting online gambling platforms, with sections titled \'十大网赌平台推荐\' (Top 10 Online Gaming Platforms Recommendation) and \'领导\' (Leadership), which suggests a focus on gaming-related content. The presence of keywords such as \'克里斯的角落\' (Chris\'s Corner) and \'十大网赌平台推荐概览\' (Overview of Top 10 Online Gaming Platforms) further supports the classification." }'

In [33]:
with open("prompts/russian.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "benign", "reason": "The website https://fadm.gov.ru appears to be an official government agency website for the Russian Federation\'s Agency for Youth Affairs (Росмолодёжь). The content is focused on youth development, education, and community engagement. The site does not contain any explicit or mature themes." }'

In [21]:
with open("prompts/ukrainian.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "Benign", "reason": "The website https://zyrixe.pp.ua appears to be a Ukrainian-language personal blog or website, containing various articles and posts on topics such as personal life, relationships, health, technology, and entertainment. The content is general in nature and does not contain explicit sexual content, violence, or illegal activities." }'

In [22]:
with open("prompts/italian.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "Benign", "reason": "The website https://www.starbenekr.it appears to be a medical center\'s official website, providing information about the services and staff. The content is primarily text-based and does not contain any explicit or adult material. The categorization is based on the absence of keywords typically associated with gambling or pornography." }'

In [23]:
with open("prompts/portuguese.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "Benign", "reason": "The website https://russelvagas.com.br appears to be a job search platform or a news portal, as it contains sections for posting job vacancies and searching for candidates. The content also includes links to YouTube videos and articles on various topics. There is no explicit or suggestive content present." }'

In [24]:
with open("prompts/dutch.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "Benign", "reason": "The website https://landjevandeboer.nl appears to be a cultural event platform for the Landje van De Boer organization. The content is focused on poetry readings and natural themes, with no explicit or mature content detected." }'

In [28]:
with open("prompts/french.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "Benign", "reason": "The website https://lesateliersimaginaires.com appears to be a community-driven forum for independent role-playing games, with features such as user registration, password recovery, and various sections for discussion and library access. The language used is French, and the content does not contain any explicit or suggestive material, indicating that it is a benign website." }'

In [30]:
with open("prompts/hungarian.txt", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": system_prompt
    },
    {
        "role": "assistant",
        "content": system_prompt_answer
    },
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{ "answer": 0, "classification": "Benign", "reason": "The website https://ielectro.hu appears to be an electronics store offering various products such as cables, connectors, and audio equipment. The content is primarily focused on product descriptions, prices, and customer reviews. There is no explicit sexual content, gambling, or adult material present." }'

In [8]:
with open("prompts\hallucinations\developer.broadcom.com", "r") as file:
    input = file.read()

conversation = [
    {
        "role": "user",
        "content": labelling_prompt
    },
    {
        "role": "assistant",
        "content": labelling_prompt_answer
    },
    {
        "role": "user",
        "content": input_suffix + input
    }
]

send_conversation_to_model(chat_url, model, conversation)

'{\n  "answer": 0,\n  "classification": "Benign",\n  "reason": "The website https://developer.broadcom.com is a developer portal for Broadcom, which contains information about APIs and related resources. There are no signs of gambling or pornographic content."\n}\n  \n\n \n  \n\n  \n  \n\n  \n  \n\n  \n  \n\n  \n  \n\n  \n  \n\n  \n  \n\n  '

In [6]:
# Prepare static system prompts
with open("prompts/system_prompt.txt", "r") as file:
    system_prompt = file.read()

with open("prompts/system_prompt_answer.txt", "r") as file:
    system_prompt_answer = file.read()

with open("prompts/labelling_promptv2.txt", "r") as file:
    labelling_prompt = file.read()

with open("prompts/labelling_prompt_answer.txt", "r") as file:
    labelling_prompt_answer = file.read()

input_suffix = "Classify the given URL as 0 (benign), 1 (gambling), or 2 (pornography) and respond with the result in JSON format.\n"

In [14]:
MAX_CONTENT_LENGTH = 1750

In [31]:
# Load the CSV file
input_file = "../domainhanan_and_content.csv"
df = pd.read_csv(input_file)
print(MAX_CONTENT_LENGTH)
output_file = "../domainhanan_and_content_label.csv"

# Function to safely extract JSON values
def extract_json_value(json_obj, key, default_value=None):
    try:
        return json_obj.get(key, default_value)
    except:
        return default_value

# Iterate over each row of the CSV
for index, row in df.iterrows():
    domain = row['Domain']
    content = row['Content']

    # Apply the sampling method
    sampled_content = sample_content(content, MAX_CONTENT_LENGTH)

    # Create input text by appending the domain content
    input_text = input_suffix + domain + ",'" + sampled_content + "'"

    # Construct the conversation payload
    conversation = [
        {"role": "user", "content": labelling_prompt},
        {"role": "assistant", "content": labelling_prompt_answer},
        {"role": "user", "content": input_text}
    ]


    # Send the conversation to the model
    start_time = time.time()
    model_response = send_conversation_to_model(chat_url, model, conversation)
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Strip spaces, newlines, and extra whitespaces
    cleaned_response = model_response.replace('\n', '').replace('\r', '').strip()

    # Print the elapsed time, domain, and cleaned response
    print(f"{elapsed_time} {domain} {cleaned_response}")

    # Parse the cleaned_response as JSON
    try:
        response_json = json.loads(cleaned_response)
    except json.JSONDecodeError:
        print(f"Error decoding JSON for domain {domain}: {cleaned_response}")
        response_json = {}

    # Extract values from the JSON response
    answer = extract_json_value(response_json, 'answer', -1)
    classification = extract_json_value(response_json, 'classification', 'Unknown')
    reason = extract_json_value(response_json, 'reason', 'No reason provided')
    
    # Update the 'Label' column with the model's answer
    df.at[index, 'Label'] = answer

# Save the updated DataFrame back to CSV
output_file = "path/to/domainhanan_and_content_with_labels.csv"
df.to_csv(output_file, index=False)

print(f"Process completed. The updated file is saved to {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: '../domainhanan_and_content.csv'

# Label

In [11]:
import json
import time
import csv
import sys

# Increase the CSV field size limit to handle large content fields
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

input_file          = r"F:\SKRIPSI\Data\0-49\Processed\domains_and_content_processed.csv"
output_file         = r"F:\SKRIPSI\Data\0-49\Processed\domains_and_content_processed_label6.csv"
output_fixed_file   = r"F:\SKRIPSI\Data\0-49\Processed\domains_and_content_processed_label6.csv"
input_suffix = "Classify the given URL as 0 (benign), 1 (gambling), or 2 (pornography) and respond with the result in JSON format.\n"
# input_suffix = "EXPECTED_CLASSIFICATION = {0: \"Benign\", 1: \"Gambling\", 2: \"Pornography\"}. Respond with the result in JSON format.\n"
MAX_CONTENT_LENGTH  = 1750  # Max content length for sampling
start_row = 7729+10251+4507+5532  # Specify the row number to start processing from

# Function to safely extract JSON values
def extract_json_value(json_obj, key, default_value=None):
    try:
        return json_obj.get(key, default_value)
    except:
        return default_value

# Open both output files for writing
with open(output_file, mode='w', newline='', encoding='utf-8') as label_csvfile, \
     open(output_fixed_file, mode='w', newline='', encoding='utf-8') as fixed_csvfile:
    
    # Prepare CSV writers
    label_fieldnames = ['Domain', 'Answer', 'Classification', 'Reason']
    fixed_fieldnames = ['Domain', 'Content', 'Label']
    label_writer = csv.DictWriter(label_csvfile, fieldnames=label_fieldnames)
    fixed_writer = csv.DictWriter(fixed_csvfile, fieldnames=fixed_fieldnames)
    
    # Write headers for both output files
    label_writer.writeheader()
    fixed_writer.writeheader()

    # Read the input CSV file row by row
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)

        # Skip rows until the start_row is reached
        for i in range(start_row):
            next(reader)
        
        for row in reader:
            domain = row['Domain']
            content = row['Content']
            
            sampled_content = sample_content(content, MAX_CONTENT_LENGTH)

            # Create input text by appending the domain and sampled content
            input_text = input_suffix +  domain + ",\"" + sampled_content + "\""

            conversation = [
                {"role": "user", "content": labelling_prompt},
                {"role": "assistant", "content": labelling_prompt_answer},
                {"role": "user", "content": input_text}
            ]

            start_time = time.time()
            model_response = send_conversation_to_model(chat_url, model, conversation)
            end_time = time.time()
            elapsed_time = end_time - start_time

            cleaned_response = model_response.replace('\n', '').replace('\r', '').strip()

            try:
                response_json = json.loads(cleaned_response)
            except json.JSONDecodeError:
                print(f"Error decoding JSON for domain {domain}: {cleaned_response}")
                response_json = {}

            answer = extract_json_value(response_json, 'answer', -1)
            classification = extract_json_value(response_json, 'classification', 'Unknown')
            reason = extract_json_value(response_json, 'reason', 'No reason provided')

            label_writer.writerow({
                'Domain': domain,
                'Answer': answer,
                'Classification': classification,
                'Reason': reason
            })
            fixed_writer.writerow({
                'Domain': domain,
                'Content': content,
                'Label': answer
            })
            # print(input_text) # DEBUGGING
            
            print(f"{elapsed_time} {domain} {cleaned_response}")

print(f"Process complete. The new CSV files have been saved to {output_file} and {output_fixed_file}")


2.107473850250244 https://dev.innovate.co.at {  "answer": 0,  "classification": "Benign",  "reason": "The website https://dev.innovate.co.at appears to be a product page for LED lights, specifically 'INNOVATE Einbaustrahler BRUSH TWO, 2fach eckig, Alu, schwenkbar'. The content includes information about the product and its features without any explicit sexual or gambling references."}
1.6306421756744385 https://dev.know-how-to-cook.com {  "answer": 0,  "classification": "Benign",  "reason": "The website https://dev.know-how-to-cook.com contains a recipe for a salad with spinach, rocket, and avocado. The content does not contain any explicit sexual or gambling material."}
1.4499797821044922 https://dev.tgifridays.com {  "answer": 0,  "classification": "Benign",  "reason": "The website https://dev.tgifridays.com appears to be a development site for TGI Fridays. The content snippet mentions 'Event Form Contact Option' and 'TGI Fridays Restaurant & Bar', which are not related to gambling o

KeyboardInterrupt: 

In [12]:
import json
import time
import csv
import sys

# Increase the CSV field size limit to handle large content fields
maxInt = sys.maxsize

while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

input_file          = r"F:\SKRIPSI\Data\0-49\Processed\domains_and_content_processed.csv"
output_file         = r"F:\SKRIPSI\Data\0-49\Processed\domains_and_content_processed_label6.csv"
output_fixed_file   = r"F:\SKRIPSI\Data\0-49\Processed\domains_and_content_processed_fixed6.csv"
input_suffix = "Classify the given URL as 0 (benign), 1 (gambling), or 2 (pornography) and respond with the result in JSON format.\n"
# input_suffix = "EXPECTED_CLASSIFICATION = {0: \"Benign\", 1: \"Gambling\", 2: \"Pornography\"}. Respond with the result in JSON format.\n"
MAX_CONTENT_LENGTH  = 1750  # Max content length for sampling
start_row = 7729+10251+4507+5532  # Specify the row number to start processing from

# Function to safely extract JSON values
def extract_json_value(json_obj, key, default_value=None):
    try:
        return json_obj.get(key, default_value)
    except:
        return default_value

# Open both output files for writing
with open(output_file, mode='w', newline='', encoding='utf-8') as label_csvfile, \
     open(output_fixed_file, mode='w', newline='', encoding='utf-8') as fixed_csvfile:
    
    # Prepare CSV writers
    label_fieldnames = ['Domain', 'Answer', 'Classification', 'Reason']
    fixed_fieldnames = ['Domain', 'Content', 'Label']
    label_writer = csv.DictWriter(label_csvfile, fieldnames=label_fieldnames)
    fixed_writer = csv.DictWriter(fixed_csvfile, fieldnames=fixed_fieldnames)
    
    # Write headers for both output files
    label_writer.writeheader()
    fixed_writer.writeheader()

    # Read the input CSV file row by row
    with open(input_file, mode='r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)

        # Skip rows until the start_row is reached
        for i in range(start_row):
            next(reader)
        
        for row in reader:
            domain = row['Domain']
            content = row['Content']
            
            sampled_content = sample_content(content, MAX_CONTENT_LENGTH)

            # Create input text by appending the domain and sampled content
            input_text = input_suffix +  domain + ",\"" + sampled_content + "\""

            conversation = [
                {"role": "user", "content": labelling_prompt},
                {"role": "assistant", "content": labelling_prompt_answer},
                {"role": "user", "content": input_text}
            ]

            start_time = time.time()
            model_response = send_conversation_to_model(chat_url, model, conversation)
            end_time = time.time()
            elapsed_time = end_time - start_time

            cleaned_response = model_response.replace('\n', '').replace('\r', '').strip()

            try:
                response_json = json.loads(cleaned_response)
            except json.JSONDecodeError:
                print(f"Error decoding JSON for domain {domain}: {cleaned_response}")
                response_json = {}

            answer = extract_json_value(response_json, 'answer', -1)
            classification = extract_json_value(response_json, 'classification', 'Unknown')
            reason = extract_json_value(response_json, 'reason', 'No reason provided')

            label_writer.writerow({
                'Domain': domain,
                'Answer': answer,
                'Classification': classification,
                'Reason': reason
            })
            fixed_writer.writerow({
                'Domain': domain,
                'Content': content,
                'Label': answer
            })
            # print(input_text) # DEBUGGING
            
            print(f"{elapsed_time} {domain} {cleaned_response}")

print(f"Process complete. The new CSV files have been saved to {output_file} and {output_fixed_file}")


1.883033037185669 https://dev.innovate.co.at {  "answer": 0,  "classification": "Benign",  "reason": "The website https://dev.innovate.co.at appears to be an online store for LED products, specifically for a type of lighting called 'INNOVATE Einbaustrahler BRUSH TWO.' The content snippet does not contain any explicit sexual content or gambling-related material."}
1.6093370914459229 https://dev.know-how-to-cook.com {  "answer": 0,  "classification": "Benign",  "reason": "The website https://dev.know-how-to-cook.com contains a recipe for a salad with spinach, rocket, and avocado. The content does not contain any explicit sexual or gambling material."}
1.4827618598937988 https://dev.tgifridays.com {  "answer": 0,  "classification": "Benign",  "reason": "The website https://dev.tgifridays.com is a development site for TGI Fridays Restaurant & Bar. The content snippet does not contain any explicit sexual material or gambling-related information. It appears to be related to a password-protec

KeyboardInterrupt: 